In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bertviz.representation_viz import RepresentationData, VerbVisualizationData
from bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer
from bertviz.save_embeddings import save_embeddings

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [33]:
#bert_version = '../UCSD-NLP/entity_normalization/data/embeddings/pubmed_pmc_470k/biobert_pytorch/'
bert_version = 'bert-base-uncased'
model = BertModel.from_pretrained(bert_version)
tokenizer = BertTokenizer.from_pretrained(bert_version)
sentences = ['I ran to the store.', 'I ran for president.', 'I ran for congress.']

02/19/2019 11:42:30 - INFO - bertviz.pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at /Users/dbw003/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
02/19/2019 11:42:30 - INFO - bertviz.pytorch_pretrained_bert.modeling -   extracting archive file /Users/dbw003/.pytorch_pretrained_bert/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /var/folders/tt/6fkylh2177j2425qfx281qmm0000gp/T/tmp_qm3hj2j
02/19/2019 11:42:33 - INFO - bertviz.pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_at

In [34]:
rep_data = RepresentationData(model, tokenizer)
toks_all = []
reps_all = []

for sent in sentences:
    tokens, reps = rep_data.get_data(sent)
    toks_all.append(tokens)
    reps_all.append(reps)

In [35]:
from collections import defaultdict
import numpy as np

embs = defaultdict(lambda: [None, [], [], []])
for j,(reps, tokens) in enumerate(zip(reps_all, toks_all)):
    for i,layer in enumerate(reps):
        embs['layer%d'%i][1].append(layer)
        embs['layer%d'%i][2].extend(tokens)
        embs['layer%d'%i][3].extend([j] * len(tokens))
        #embs['layer%d'%i].extend([None, layer, tokens, [j] * len(tokens)])
for i,layer in enumerate(reps):
    embs['layer%d'%i][1] = np.vstack(embs['layer%d'%i][1])

In [36]:
save_embeddings(embs, './hidden_viz')

# Verb clustering visualization

In [6]:
import pandas as pd
import random
import itertools

snippets = pd.read_csv('./verb_snippets.txt', sep='\t')
samples = snippets.iloc[random.sample(range(snippets.shape[0]), 1000)]
sentences = samples.values[:,3]
bacteria_locations = [[s.split(':')[0] for s in sample[4:-2].split(' ')] for sample in samples.values[:,1]]
disease_locations = [[s.split(':')[0] for s in sample[4:-2].split(' ')] for sample in samples.values[:,2] ]
boundaries = []
for bac,dis in zip(bacteria_locations, disease_locations):
    locations = [tuple(sorted([int(k) for k in t])) for t in list(itertools.product(bac,dis))]
    boundaries.append(locations)


bert_version = '../UCSD-NLP/entity_normalization/data/embeddings/pubmed_pmc_470k/biobert_pytorch/'
#bert_version = 'bert-base-uncased'
model = BertModel.from_pretrained(bert_version)
tokenizer = BertTokenizer.from_pretrained(bert_version)

02/21/2019 12:54:46 - INFO - bertviz.pytorch_pretrained_bert.modeling -   loading archive file ../UCSD-NLP/entity_normalization/data/embeddings/pubmed_pmc_470k/biobert_pytorch/
02/21/2019 12:54:46 - INFO - bertviz.pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

02/21/2019 12:54:48 - INFO - bertviz.pytorch_pretrained_bert.tokenization -   loading vocabulary file ../UCSD-NLP/entity_normalization/data/embeddings/pubmed_pmc_470k/biobert_pytorch/vocab.txt


In [7]:
len(boundaries)

1000

In [5]:
rep_data = VerbVisualizationData(model, tokenizer)
toks_all = []
reps_all = []
groups_all = []

for sent,bounds in zip(sentences, boundaries):
    tokens, reps, groups = rep_data.get_data(sent, bounds)
    toks_all.append(tokens)
    reps_all.append(reps)
    groups_all.append(groups)

In [6]:
from collections import defaultdict
import numpy as np

embs = defaultdict(lambda: [None, [], [], [], []])
for j,(reps, tokens, groups) in enumerate(zip(reps_all, toks_all, groups_all)):
    for i,layer in enumerate(reps):
        embs['layer%d'%i][1].append(layer) #Tensor
        embs['layer%d'%i][2].extend(tokens) #Tokens
        embs['layer%d'%i][3].extend(groups) #Group ID
        embs['layer%d'%i][4].extend([j] * len(tokens)) #Sentence ID
        #embs['layer%d'%i].extend([None, layer, tokens, [j] * len(tokens)])
for i,layer in enumerate(reps):
    embs['layer%d'%i][1] = np.vstack(embs['layer%d'%i][1])

In [7]:
save_embeddings(embs, './verb_viz')

# Saving off the verb vectors for clustering

In [3]:
import pandas as pd
import random
import itertools
import numpy as np
from tqdm import tqdm_notebook

samples = pd.read_csv('./verb_snippets.txt', sep='\t')
sentences = samples.values[:,3]
bacteria_locations = [[s.split(':')[0] for s in sample[4:-2].split(' ')] for sample in samples.values[:,1]]
disease_locations = [[s.split(':')[0] for s in sample[4:-2].split(' ')] for sample in samples.values[:,2] ]
boundaries = []
for bac,dis in zip(bacteria_locations, disease_locations):
    locations = [tuple(sorted([int(k) for k in t])) for t in list(itertools.product(bac,dis))]
    boundaries.append(locations)


bert_version = '../UCSD-NLP/entity_normalization/data/embeddings/pubmed_pmc_470k/biobert_pytorch/'
#bert_version = 'bert-base-uncased'
model = BertModel.from_pretrained(bert_version)
tokenizer = BertTokenizer.from_pretrained(bert_version)

02/21/2019 12:56:55 - INFO - bertviz.pytorch_pretrained_bert.modeling -   loading archive file ../UCSD-NLP/entity_normalization/data/embeddings/pubmed_pmc_470k/biobert_pytorch/
02/21/2019 12:56:55 - INFO - bertviz.pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 28996
}

02/21/2019 12:56:57 - INFO - bertviz.pytorch_pretrained_bert.tokenization -   loading vocabulary file ../UCSD-NLP/entity_normalization/data/embeddings/pubmed_pmc_470k/biobert_pytorch/vocab.txt


In [24]:
pmids = samples.values[:,0]

In [17]:
import tables
import numpy as np

filename = './verb_clustering/vectors.h5'
ROW_SIZE = 768
f = tables.open_file(filename, mode='w')
atom = tables.Float32Atom()
array_c = f.create_earray(f.root, 'data', atom, (0, ROW_SIZE))
f.close()

with open('./verb_clustering/metadata.tsv', 'w') as f:
    f.write("index\tverb\tsentence\n")
    
rep_data = VerbVisualizationData(model, tokenizer)
reps_all = []
verbs_all = []
idxs_all = []
sentences_all = []

In [18]:
start_index = len([l for l in open('./verb_clustering/metadata.tsv')]) - 1
print("Starting from index %d"%(start_index))

for i, (sent,bounds) in enumerate(tqdm_notebook(zip(sentences[start_index:], boundaries[start_index:]), total=len(sentences[start_index:]))):
    tokens, reps, groups, verb_idxs, bert_verb_idxs = rep_data.get_data(sent, bounds)
    for idx,bidx in zip(verb_idxs, bert_verb_idxs):
        sentences_all.append(sent)
        reps_all.append(reps[-1,bidx,:])
        verbs_all.append(tokens[bidx])
        idxs_all.append(idx)
    
    #Save off every 10000 iterations since this takes 5ever
    if (i+1) % 10000 == 0:
        f = tables.open_file(filename, mode='a')
        for x in reps_all:
            f.root.data.append(np.expand_dims(x, 0))
        f.close()
        reps_all = []
        
        with open('./verb_clustering/metadata.tsv', 'a+') as f:
            for idx,verb,phrase in zip(idxs_all, verbs_all, sentences_all):
                f.write("\t".join([str(idx), verb, phrase]) + '\n')
        idxs_all = []
        verbs_all = []
        sentences_all = []

#Finish   
f = tables.open_file(filename, mode='a')
for x in reps_all:
    f.root.data.append(np.expand_dims(x, 0))
f.close()
reps_all = []

with open('./verb_clustering/metadata.tsv', 'a+') as f:
    for idx,verb,phrase in zip(idxs_all, verbs_all, sentences_all):
        f.write("\t".join([str(idx), verb, phrase]) + '\n')
idxs_all = []
verbs_all = []
sentences_all = []

Starting from index 0


ValueError: the ranks of the appended object (1) and the ``/data`` EArray (2) differ

In [22]:
#Finish   
f = tables.open_file(filename, mode='a')
for x in reps_all:
    f.root.data.append(np.expand_dims(x, 0))
f.close()
reps_all = []

with open('./verb_clustering/metadata.tsv', 'a+') as f:
    for idx,verb,phrase in zip(idxs_all, verbs_all, sentences_all):
        f.write("\t".join([str(idx), verb, phrase]) + '\n')
idxs_all = []
verbs_all = []
sentences_all = []

In [21]:
reps_all[0].shape

(768,)

In [28]:
pmid_list = []
for i, (sent,bounds) in enumerate(tqdm_notebook(zip(sentences[start_index:], boundaries[start_index:]), total=len(sentences[start_index:]))):
    tokens, reps, groups, verb_idxs, bert_verb_idxs = rep_data.get_data(sent, bounds)
    for idx,bidx in zip(verb_idxs, bert_verb_idxs):
        pmid_list.append(pmids[i])

In [23]:
len(sentences)

664260

In [29]:
len(pmid_list)

253188

In [30]:
start_index

0

In [32]:
sent_to_pmid = {}
for v in samples.values:
    if v[3] in sent_to_pmid:
        del sent_to_pmid[v[3]]
    else:
        sent_to_pmid[v[3]] = v[0]

In [33]:
len(sent_to_pmid)

661846

In [35]:
metadata = pd.read_csv('./verb_clustering/metadata.tsv', sep='\t')

In [36]:
metadata.insert(0, 'PMID', pmid_list)

In [37]:
metadata.to_csv('./verb_clustering/metadata.tsv', sep='\t', index=None)

In [38]:
len(reps_all)

0